# Weeks 9 Exercises

Name: Kesav Adithya Venkidusamy <br>
Course: DSC550 - Data Mining <br>
Instructor: Brett Werner <br>

## Best Model Selection and Hyperparameter Tuning

##### Importing libraries

In [1]:
## Importing the libraries required for this assignment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #used to split data into training/test sets
from sklearn.metrics import plot_confusion_matrix
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from yellowbrick.classifier import ConfusionMatrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

##### 1. Import the dataset and ensure that it loaded properly.

In [6]:
#load the data into pandas dataframe
loan_df = pd.read_csv('Loan_Train.csv') 
loan_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [7]:
## Check the shape of the dataframe
loan_df.shape

(614, 13)

In [8]:
## Printing the datatypes of each column
loan_df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

##### 2: Prepare the data for modeling by performing the following steps:

###### Drop the column “Load_ID.”

In [10]:
## Dropping the Load_ID and showing few records
loan_df = load_df.drop(['Loan_ID'], axis=1)
loan_df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [11]:
## Display the shape of dataframe
loan_df.shape

(614, 12)

##### Drop any rows with missing data.

In [12]:
## Drop rows with missing data using dropna() function  
loan_df = loan_df.dropna()

In [13]:
## check shape of df to ensure rows dropped
loan_df.shape

(480, 12)

In [14]:
## Showing few records in the dataframe
loan_df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


##### Convert the categorical features into dummy variables.

In [15]:
## Summarize the categorical columns in the dataframe
catCols = loan_df.select_dtypes("object").columns
catCols= list(set(catCols))
catCols

['Married',
 'Property_Area',
 'Dependents',
 'Gender',
 'Loan_Status',
 'Self_Employed',
 'Education']

##### New Column creation for Loan Status

In [21]:
## Create a new column that converts the class to a coded class of 1 and 0 (Y=1 and N=0) for target in model
loan_df['Loan_Status_Coded'] = loan_df['Loan_Status'].replace(to_replace=["Y","N"], value=[1,0])

In [22]:
## Create dummy variables for Categorical columns
loan_dummy_df = pd.get_dummies(loan_df, columns=catCols)

In [23]:
## Showing the columns and dtypes
loan_dummy_df.dtypes

ApplicantIncome              int64
CoapplicantIncome          float64
LoanAmount                 float64
Loan_Amount_Term           float64
Credit_History             float64
Loan_Status_Coded            int64
Married_No                   uint8
Married_Yes                  uint8
Property_Area_Rural          uint8
Property_Area_Semiurban      uint8
Property_Area_Urban          uint8
Dependents_0                 uint8
Dependents_1                 uint8
Dependents_2                 uint8
Dependents_3+                uint8
Gender_Female                uint8
Gender_Male                  uint8
Loan_Status_N                uint8
Loan_Status_Y                uint8
Self_Employed_No             uint8
Self_Employed_Yes            uint8
Education_Graduate           uint8
Education_Not Graduate       uint8
dtype: object

##### 3: Split the data into a training and test set, where the “Loan_Status” column is the target.

In [26]:
## split the data into a training and test set
## drop the target variables related to Loan_Status
## get the target
X = loan_dummy_df.drop(['Loan_Status_Coded', 'Loan_Status_N', 'Loan_Status_Y'], axis = 1)
y = loan_dummy_df['Loan_Status_Coded'] # (Y=1 and N=0)

## split the data into training and test sets (80% Training/20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## Reset indices in the training and test sets to prevent pandas slicing warnings
X_train = X_train.reset_index(drop = True) #drop + True drops the previous index
X_test = X_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [54]:
## show the sizes of the training and test sets
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(384, 20)
(96, 20)
(384,)
(96,)


In [29]:
## Number of records for Loan Status: (Y=1 and N=0)
print(y_train.value_counts())
print(y_test.value_counts())

1    264
0    120
Name: Loan_Status_Coded, dtype: int64
1    68
0    28
Name: Loan_Status_Coded, dtype: int64


In [30]:
## view the training features
X_train.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Married_No,Married_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Gender_Female,Gender_Male,Self_Employed_No,Self_Employed_Yes,Education_Graduate,Education_Not Graduate
0,4350,0.0,154.0,360.0,1.0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,1
1,3315,0.0,96.0,360.0,1.0,0,1,0,1,0,0,1,0,0,0,1,1,0,1,0
2,1963,0.0,53.0,360.0,1.0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1
3,5568,2142.0,175.0,360.0,1.0,0,1,1,0,0,1,0,0,0,0,1,1,0,1,0
4,4124,0.0,115.0,360.0,1.0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0


#####  4: Create a pipeline with a min-max scaler and a KNN classifier (see section 15.3 in the Machine Learning with Python Cookbook).

In [31]:
## Loading required libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

In [32]:
## Create minmax scaler
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,1))

In [33]:
## Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

In [34]:
## Create pipeline
pipe= Pipeline([("scaler", minmax_scale), ("knn", knn)])

##### 5. Fit a default KNN classifier to the data with this pipeline. Report the model accuracy on the test set. Note: Fitting a pipeline model works just like fitting a regular model.

In [35]:
## Fit a default KNN classifier to the data with the pipeline
pipe.fit(X_train, y_train) 

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('knn', KNeighborsClassifier(n_jobs=-1))])

In [36]:
## Predict Output
y_pred = pipe.predict(X_test)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1], dtype=int64)

In [37]:
## Report model accuracy on the test set
pipe.score(X_test,y_test)

0.7083333333333334

##### 6. Create a search space for your KNN classifier where your “n_neighbors” parameter varies from 1 to 10. (see section 15.3 in the Machine Learning with Python Cookbook)

In [47]:
## Create space of candidate values
search_space = [{"knn__n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}]

##### 7. Fit a grid search with your pipeline, search space, and 5-fold cross-validation to find the best value for the “n_neighbors” parameter.

In [48]:
## Create standardizer
standardizer = StandardScaler()

In [49]:
## Standardize features
X_standardized = standardizer.fit_transform(X)

In [50]:
## Create grid search
classifier = GridSearchCV(pipe, search_space, cv=5, verbose=0).fit(X_standardized, y)

In [52]:
## Best neighborhood size (k)
classifier.best_estimator_.get_params()["knn__n_neighbors"]

3

##### 8. Find the accuracy of the grid search best model on the test set. Note: It is possible that this will not be an improvement over the default model, but likely it will be.

In [58]:
print("KNN tuned hpyerparameters: {}".format(classifier.best_params_))
print("KNN accuracy: {}".format(classifier.best_score_))
print("KNN Accuracy percentage: {:.2f}%".format(classifier.best_score_*100))

KNN tuned hpyerparameters: {'knn__n_neighbors': 3}
KNN accuracy: 0.7229166666666667
KNN Accuracy percentage: 72.29%


#### 9. Now, repeat steps 6 and 7 with the same pipeline, but expand your search space to include logistic regression and random forest models with the hyperparameter values in section 12.3 of the Machine Learning with Python Cookbook.

In [60]:
## Re-create pipeline with KNN hyperparameters
pipe= Pipeline([('standardizer', StandardScaler()), ("classifier", KNeighborsClassifier(n_neighbors=5, n_jobs=-1))])

In [61]:
## Expand search_space to include logistic regression and random forest models
## Create dictionary with candidate learning algorithms and their hyperparameters
search_space=[
    {'classifier' : [KNeighborsClassifier()], 
     'classifier__n_neighbors' : [1,2,3,4,5,6,7,8,9,10]},
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(0, 4, 10),
    'classifier__solver' : ['liblinear']},
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : [10,100,1000],
    'classifier__max_features' : [1, 2, 3]}
]

In [62]:
## Create grid search
gridsearch=GridSearchCV(pipe, search_space, cv = 5, verbose=0)

In [63]:
## Fit grid search
best_model=gridsearch.fit(X_train, y_train)

###### 10. What are the best model and hyperparameters found in the grid search? Find the accuracy of this model on the test set.

In [64]:
## Best neighborhood size (k)
best_model.best_estimator_.get_params()
## The best model is the Logistic Regression Model with the hyperparameters listed below

{'memory': None,
 'steps': [('standardizer', StandardScaler()),
  ('classifier', LogisticRegression(solver='liblinear'))],
 'verbose': False,
 'standardizer': StandardScaler(),
 'classifier': LogisticRegression(solver='liblinear'),
 'standardizer__copy': True,
 'standardizer__with_mean': True,
 'standardizer__with_std': True,
 'classifier__C': 1.0,
 'classifier__class_weight': None,
 'classifier__dual': False,
 'classifier__fit_intercept': True,
 'classifier__intercept_scaling': 1,
 'classifier__l1_ratio': None,
 'classifier__max_iter': 100,
 'classifier__multi_class': 'auto',
 'classifier__n_jobs': None,
 'classifier__penalty': 'l2',
 'classifier__random_state': None,
 'classifier__solver': 'liblinear',
 'classifier__tol': 0.0001,
 'classifier__verbose': 0,
 'classifier__warm_start': False}

In [65]:
# Fit grid search
best_model=gridsearch.fit(X_test, y_test)

In [67]:
print("Best Model accuracy:",best_model.best_score_)
print("Best Model Accuracy percentage: {:.2f}%".format(best_model.best_score_*100))

Best Model accuracy: 0.7494736842105263
Best Model Accuracy percentage: 74.95%


##### 11. Summarize your results.

<b>The Logistic Regression with the hyperparameters above is the best model to use for this dataset.</b>